In [2]:
# Importação das bibliotecas necessárias
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Definindo o número de pacientes na base de dados de validação externa
num_patients = 196

# Probabilidades para cada local primário do tumor (Primary tumor site)
# A soma das probabilidades deve ser exatamente 1
primary_tumor_site_probabilities = [0.062, 0.052, 0.3, 0.02, 0.15, 0.05, 0.12, 0.114, 0.022, 0.11]
print("Soma das probabilidades de Primary tumor site:", sum(primary_tumor_site_probabilities))

# Gerando a base de dados fictícia
# A seguir, criamos um dicionário onde cada chave representa uma variável (coluna) da base de dados
data = {
    # Idade dos pacientes (Age): utilizando uma distribuição normal com média de 65 anos e desvio padrão de 10 anos
    # Isso simula idades típicas de pacientes em uma faixa etária avançada, arredondadas para números inteiros
    "Age (years)": np.random.normal(loc=65, scale=10, size=num_patients).astype(int),

    # Gênero dos pacientes (Gender): escolhendo entre "Female" e "Male" com as probabilidades fornecidas
    "Gender": np.random.choice(["Female", "Male"], size=num_patients, p=[0.449, 0.551]),

    # Raça dos pacientes (Race): escolhendo entre "White", "Black", e "Other" com as probabilidades fornecidas
    "Race": np.random.choice(["White", "Black", "Other"], size=num_patients, p=[0.77, 0.09, 0.14]),

    # Origem hispânica (Hispanic): indicando se o paciente é hispânico ou não
    "Hispanic": np.random.choice(["No", "Yes"], size=num_patients, p=[0.92, 0.08]),

    # Estado civil dos pacientes (Marital status): escolhendo entre "Married", "Single", e "Unknown"
    "Marital status": np.random.choice(["Married", "Single", "Unknown"], size=num_patients, p=[0.41, 0.53, 0.06]),

    # Tipo histológico do tumor (Histological type): escolhendo entre três tipos de células
    "Histological type": np.random.choice(["Adenocarcinoma", "Mucinous", "Signet ring cell carcinoma"], size=num_patients, p=[0.83, 0.14, 0.03]),

    # Grau do tumor (Grade): representando a diferenciação do tumor em quatro graus, além de "Unknown"
    "Grade": np.random.choice(["Grade I", "Grade II", "Grade III", "Grade IV", "Unknown"], size=num_patients, p=[0.133, 0.725, 0.122, 0.015, 0.005]),

    # Localização primária do tumor (Primary tumor site): distribuída entre 10 possíveis locais
    "Primary tumor site": np.random.choice(
        ["Cecum", "Appendix", "Ascending colon", "Hepatic flexure of colon", "Transverse colon",
         "Splenic flexure of colon", "Descending colon", "Sigmoid colon", "Rectum", "Overlapping lesion of colon"],
        size=num_patients,
        p=primary_tumor_site_probabilities
    ),

    # Estágio T do tumor (T stage): representando a extensão do tumor
    "T stage": np.random.choice(["T1", "T2", "T3", "T4", "TX"], size=num_patients, p=[0.031, 0.087, 0.658, 0.224, 0.0]),

    # Estágio N do tumor (N stage): representando a extensão de comprometimento linfático
    "N stage": np.random.choice(["N0", "N1", "N2", "NX"], size=num_patients, p=[0.571, 0.286, 0.143, 0.0]),

    # Tamanho do tumor em centímetros (Tumor size): valores contínuos baseados em proporções
    # Aproximadamente 50.5% dos tumores são <5 cm, 34.2% >=5 cm, e 16.2% são "Unknown" (NaN)
    "Tumor size (cm)": np.where(
        np.random.rand(num_patients) < 0.505,  # 50.5% dos tumores serão <5 cm
        np.random.normal(loc=3, scale=1, size=num_patients),  # Tamanho médio de 3 cm para tumores <5 cm
        np.where(
            np.random.rand(num_patients) < 0.342 / (1 - 0.505),  # 34.2% dos tumores restantes são >=5 cm
            np.random.normal(loc=7, scale=1.5, size=num_patients),  # Tamanho médio de 7 cm para tumores >=5 cm
            np.nan  # 16.2% dos tumores são "Unknown" (valores NaN)
        )
    ).round(2),  # Arredonda o tamanho do tumor para 2 casas decimais

    # Níveis de CEA (antígeno carcinoembrionário): indicativo da presença de antígenos tumorais
    "CEA": np.random.choice(["Negative", "Borderline", "Positive", "Unknown"], size=num_patients, p=[0.536, 0.0, 0.296, 0.168]),

    # Depósitos tumorais (Tumor deposits): indicação da presença de depósitos tumorais
    "Tumor deposits": np.random.choice(["No", "Yes", "Unknown"], size=num_patients, p=[0.91, 0.09, 0.0]),

    # Metástase em fígado e/ou pulmão (Liver and/or lung metastasis): presença de metástase
    "Liver and/or lung metastasis": np.random.choice(["No", "Yes"], size=num_patients, p=[0.825, 0.175])
}

# Criação do DataFrame com os dados ajustados
# A função pd.DataFrame cria uma tabela estruturada a partir do dicionário `data`
df = pd.DataFrame(data)

# Salvando o DataFrame em um arquivo CSV
# O arquivo é salvo no caminho especificado, sem incluir o índice
df.to_csv('/content/bdsintetico.csv', index=False)
print("Arquivo CSV salvo como 'bdsintetico.csv'")


Soma das probabilidades de Primary tumor site: 1.0
Arquivo CSV salvo como 'bdsintetico.csv'
